<a href="https://colab.research.google.com/github/mdallas1/ApplyIt_Numerical_Math/blob/main/org_meeting_06122022_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introduction**

The goal of the two exercises below are to solve a linear system using three different methods, and then to numercially approximate an integral using the composite midpoint and composite simpson rules. 

#### **Solving a Linear System**

The three methods for solving a linear system $Ax=b$ featured here are direct inversion, LU factorization, and QR factorization. Direct inversion is exactly what it sounds like. The inverse of the matrix $A$ is computed and then $x=A^{-1}b$. In an LU factorization, $A$ is factored into the product of a lower triangular matrix $L$ and an upper triangular matrix $U$. This essentially Gaussian elimination. Letting $y=Ux$, the system $Ax=b$ becomes $Ly=b$. Now one solves $Ly=b$, and then $Ux=y$ to compute $x$. Similarly, a QR factorization transforms $A$ into the product of two "simpler" matrices, but here $Q$ is an orthogonal matrix, i.e., $Q^TQ=I$, and $R$ is an upper triangular matrix. Letting $y=Rx$, one can now solve $Qy=b$, then $Rx=y$ to compute $x$. 

Python has these methods built into the SciPy library. Click the links below for more information.

[la.inv](https://numpy.org/devdocs/reference/generated/numpy.linalg.inv.html)

[la.lu_factor()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_factor.html)

[la.lu_solve()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_solve.html)

[la.qr()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.qr.html)

Further reading: 

[Numerically inverting](https://gregorygundersen.com/blog/2020/12/09/matrix-inversion/)

[LU decomposition](https://johnfoster.pge.utexas.edu/numerical-methods-book/LinearAlgebra_LU.html)

[QR decomposition](https://bvanderlei.github.io/jupyter-guide-to-linear-algebra/QR_Factorization.html)

#### **Numerical Integration**

We'll implement the composite Simpson and midpoint rules to solve an integral arising in the study of optics, and compare how fast the error increases as the number of subintervals increase. Both of these methods can be derived by integrating the appropriate [interpolating polynomial](https://en.wikipedia.org/wiki/Interpolation#Piecewise_constant_interpolation) of $f$ at the nodes. The midpoint rule arises from the degree 0 interpolant, whereas Simpson's rule comes from the quadratic interpolant. Python does have the composite simpson rule implemented as **scipy.integrate.simp**. Below we import this as **simp** (click [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.simpson.html) for more info). 

To be more concrete, suppose we want to integrate $f$ over the interval $[a,b]$. Partition $[a,b]$ into $N$ subintervals $[x_i,x_{i+1}]$, $i=0,1,...,N-1$, where $x_{i+1}-x_i = (b-a)/N=:h$.  The we get the following quadrature rules: 

**Composite Midpoint:** $\int_a^b f(x)\,dx \approx h\sum_{i=0}^{N-1} f\big(\frac{x_i+x_{i+1}}{2}\big)$

**Composite Simpson's Rule:** for $i=1,...,N-1$, 

$\int_{x_{i-1}}^{x_{i+1}} f(x)\,dx\approx \frac{h}{3} \bigg(f(x_{i-1})+4f(x_i)+f(x_{i+1})\bigg)$.

Summing over $i$ we get

$\frac{h}{3}\bigg( f(x_0)+4\sum_{i \text{ odd}}^{N-1} f(x_i)+2\sum_{i \text{ even}}^{N-2}f(x_i)+f(x_N)\bigg)$.

Further reading: 

[Composite Simpson's rule](https://pythonnumericalmethods.berkeley.edu/notebooks/chapter21.04-Simpsons-Rule.html)

[Composite Midpoint rule](http://hplgit.github.io/prog4comp/doc/pub/._p4c-solarized-Python017.html)

===================================================================================================================

## **Solving a Hilbert system**

The system we'll solve involves the so called [Hilbert matrix](https://en.wikipedia.org/wiki/Hilbert_matrix). It's a notoriously [ill-conditioned](https://en.wikipedia.org/wiki/Condition_number) matrix, which means solving systems with it can be very difficult. The point of this exercise is not to solve this system to high precision, rather the goal is to observe how the three methods mentioned above differ in performance. Starting with $n=10$, increase the dimension of the system and observe how the error changes in the three methods. 

In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
from scipy.integrate import simpson as simp

#note that numpy has an exp implementation as well
from math import exp as exp 
from time import process_time 

#--------------------------------------------------------------------------------
#set dimension of Hilbert matrix. Note how the error changes below as n increases
n = 10
#--------------------------------------------------------------------------------

#Define Hilbert matrix H
v = np.array(range(1,2*n))
v = 1/v
H = la.hankel(v[0:n],v[n-1:len(v)])

In [ ]:
#--------------------------------------------------------------------------------
#In this block a system Hx=b is setup so that the solution is x=(1,1,...,1)^T
#--------------------------------------------------------------------------------
xsol = np.ones(n) 
b = H@xsol
Hinv = la.inv(H) 
I = np.identity(n); 

In [ ]:
#--------------------------------------------------------------------------------
#In this block, solve the system by inverting H directly and print the error 
#--------------------------------------------------------------------------------

x_approx = Hinv@b;
err_x = la.norm(x_approx-xsol)/la.norm(xsol)
print("Error form inverting solve: ",err_x)

In [ ]:
#-------------------------------------------------------------------------------------------
#Solve the system using an LU factorization. You may write your own function that does this 
#or use the built in function lu_factor() and lu_solve(). These were imported with 
#"scipy.linalg as la"
#-------------------------------------------------------------------------------------------
lu,piv = la.lu_factor(H)
x_approx = la.lu_solve((lu,piv),b)
err_x = la.norm(xsol-x_approx)/la.norm(xsol)
print("LU error: ",err_x)

In [ ]:
#-------------------------------------------------------------------------------------------
#Solve by the QR factorization method. As with the LU method, you can use python's built 
#in function or write your own.
#-------------------------------------------------------------------------------------------
Q,R = la.qr(H)
Q_T = np.transpose(Q) #compute transpose
y = Q_T@b
x_approx = la.lstsq(R,y)[0]
err_x = la.norm(x_approx-xsol)/la.norm(xsol)
print("QR error: ",err_x)

===================================================================================================================

## **Numerical Integration**

This integral is taken from Quarteroni, Saleri, and Gervasio's text *Scientific Computing with MATLAB and Octave* (the pdf is free to [download](https://link.springer.com/book/10.1007/978-3-642-45367-0) at SpringerLink). Evidently it originates from the study of optics. 
    Let $T=213$. Use the composite midpoint and Simpson rules to compute the integral
\begin{align}
    E(T) = 2.39\cdot 10^{-11} \int_{3\cdot 10^{-4}}^{14\cdot 10^{-4}} \frac{dx}{x^5(e^{1.432\,/\,(Tx)}-1)}.
\end{align}
Plot the results vs number of subintervals, and compare the error as the number of subintervals increases. Use 
\begin{align*} x_{\text{sol}} = 0.0206908554816539146 \end{align*}
as the true value.

In [ ]:
f = lambda x: 1/((x**5)*(exp(1.432/(213*x))-1))
a = 3e-4
b = 14e-4
C = 2.39e-11
xsol = 0.0206908554816539146

def comp_mid(f,a,b,N):
    #-----------------------------------------------------------------------------------
    #Composite midpoint rule with N subinterval Takes as inputs a function f, the lower 
    #and upper bounds of integration a and b, and the number of subintervals N.
    #-----------------------------------------------------------------------------------
    N = int(N)
    h = (b-a)/N
    fy = [0]*(N+1)
    for i in range(0,N):
        fy[i] = f(a + (i+0.5)*h)
    return np.sum(fy)*h

def comp_simp(f,a,b,N):
    #-----------------------------------------------------------------------------------
    #Composite Simpson rule with N subinterval Takes as inputs a function f, the lower 
    #and upper bounds of integration a and b, and the number of subintervals N.
    #-----------------------------------------------------------------------------------
    N = int(N)
    h = (b-a)/N
    fy = [0]*N
    for i in range(1,N):
        if i % 2 == 0:
            fy[i] = 2*f(a+i*h)
        else:
            fy[i] = 4*f(a+i*h)
    return h/3*(np.sum(fy)+f(a)+f(b))

In [ ]:
#-----------------------------
#Plot results
#-----------------------------

t = [1,10,1e2,1e3,1e4,1e5]
n=len(t)
err_mid = [0]*n
err_simp = [0]*n
counter = 0;
for i in t:
    err_mid[counter] = la.norm(xsol-comp_mid(f,a,b,i)*C)
    err_simp[counter] = la.norm(xsol-comp_simp(f,a,b,i)*C)
    counter += 1
plt.semilogy(t,err_mid,'*-b',label = 'Comp Midpoit rule')
plt.semilogy(t,err_simp,'o-r',label = 'Comp Simpson rule')
e=[1]*n
k=0
for i in t:
    e[k]=2.22e-16
    k+=1
plt.semilogy(t,e,'--k',label = 'Machine Precision')
plt.legend();